## Load Python modules

In [1]:
import pandas as pd
from pandas.core.series import Series
import numpy as np
import glob
import datetime
# import sklearn
# import sklearn.datasets
# import sklearn.cross_validation
# import time

# from sklearn.cross_validation import KFold
# from sklearn import ensemble
# from sklearn import metrics
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
# from sklearn.cross_validation import cross_val_score
# from sklearn.tree import DecisionTreeRegressor

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
def convert_week(x):
    start_epoch_arr= x.values.astype(int)
    end_epoch_arr= np.array(start_epoch_arr)
    
    start_epoch_arr= (start_epoch_arr-400)*7+31900-6
    start_epoch_arr = (start_epoch_arr-25569) * 86400
    
    end_epoch_arr = (end_epoch_arr-400)*7+31900
    end_epoch_arr= (end_epoch_arr-25569) * 86400
    
    return (start_epoch_arr, end_epoch_arr)

def get_transaction_time(x, start_epoch_arr):
    transaction_time_epoch_arr= x.values.astype(int)
    transaction_time_epoch_arr= start_epoch_arr + (transaction_time_epoch_arr * 60)
    return transaction_time_epoch_arr

def get_datetime(transaction_time_epoch_arr):
    t_shape= np.shape(transaction_time_epoch_arr)
    month_arr= np.zeros(t_shape)
    day_arr=np.zeros(t_shape)
    year_arr= np.zeros(t_shape)
    hour_arr= np.zeros(t_shape)
    minute_arr= np.zeros(t_shape)
    dayofweek_arr= np.zeros(t_shape)
    
    for i in xrange(t_shape[0]):
        t_time= transaction_time_epoch_arr[i]
        dt= datetime.datetime.utcfromtimestamp(t_time)
        month_arr[i]= dt.month
        day_arr[i]= dt.day
        year_arr[i]= dt.year
        hour_arr[i]= dt.hour
        minute_arr[i]= dt.minute
        dayofweek_arr[i]= dt.weekday()
    
    return (month_arr, day_arr, year_arr, hour_arr, minute_arr, dayofweek_arr)

def parse_panel(filename):
    column_headers= None
    m_df_items= []
    with open(filename) as f:
        i= 0
        for line in f:
            line= line.strip()
            s_arr= line.split(",")
            s_arr= [x for x in s_arr if len(x.strip()) > 0]
            if column_headers == None:
                column_headers= s_arr
                continue
            item= (i, s_arr)
            m_df_items.append(item)
            i+=1
    if "MINUTE" not in column_headers:
        raise RuntimeError("Expecting MINUTE column in dataset, unable to parse")
        
    m_df= pd.DataFrame.from_items(m_df_items, columns=column_headers, orient='index')
    
    s_week_df= m_df["WEEK"]
    start_time_epoch_arr, end_time_epoch_arr= convert_week(s_week_df)
    r1= pd.DataFrame({
        'START_TIME_EPOCH_S' : start_time_epoch_arr
    })
    r2= pd.DataFrame({
        'END_TIME_EPOCH_S' : end_time_epoch_arr
    })
    m_df= pd.concat([m_df, r1, r2], axis=1)
    #
    s_minute_df= m_df["MINUTE"]
    transaction_time_epoch_arr= get_transaction_time(s_minute_df, start_time_epoch_arr)
    t_result= pd.DataFrame({
        'TRANSACTION_TIME_EPOCH_S' : transaction_time_epoch_arr
    })
    # extract month, day, year, hour, minute
    month_arr, day_arr, year_arr, hour_arr, minute_arr, dayofweek_arr= get_datetime(transaction_time_epoch_arr)
    r1= pd.DataFrame({
        'MONTH' : month_arr
    })
    r2= pd.DataFrame({
        'DAY' : day_arr
    })
    r3= pd.DataFrame({
        'YEAR' : year_arr
    })
    r4= pd.DataFrame({
        'HOUR_OF_DAY' : hour_arr
    })
    r5= pd.DataFrame({
        'MINUTE' : minute_arr
    })
    r6= pd.DataFrame({
        'DAYOFWEEK' : dayofweek_arr
    })
    m_df.drop("MINUTE", axis=1, inplace=True)
    m_df= pd.concat([m_df, t_result, r1, r2, r3, r4, r5, r6], axis=1)
    #
    return m_df

In [4]:
# parse panels across Year8-Year11
panel_filenames= []
panel_filenames.extend(glob.glob("/home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year8/*.DAT"))
panel_filenames.extend(glob.glob("/home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year9/*.DAT"))
panel_filenames.extend(glob.glob("/home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year10/*.DAT"))
panel_filenames.extend(glob.glob("/home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year11/*.DAT"))

master_panel_df= None
for pf in panel_filenames:
    print "Parsing {}".format(pf)
    m_df= parse_panel(pf)
    if master_panel_df is None:
        master_panel_df= m_df
    else:
        master_panel_df= pd.concat([master_panel_df, m_df], ignore_index=True)

master_panel_df.sort(["TRANSACTION_TIME_EPOCH_S"], inplace=True)
master_panel_df.reset_index(drop=True, inplace=True)
master_panel_df

Parsing /home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year8/beer_PANEL_MK_1479_1530.DAT
Parsing /home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year8/beer_PANEL_GK_1479_1530.DAT
Parsing /home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year8/beer_PANEL_KK_1479_1530.DAT
Parsing /home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year9/beer_PANEL_GK_1531_1582.DAT
Parsing /home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year9/beer_PANEL_MK_1531_1582.DAT
Parsing /home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year10/beer_PANEL_MK_1583_1634.DAT
Parsing /home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year10/beer_PANEL_GK_1583_1634.DAT
Parsing /home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year10/beer_PANEL_DK_1583_1634.DAT
Parsing /home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year11/beer_PANEL_DK_1635_1686.DAT
Parsing /home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year11/beer_PANEL_GK_1635_1686.DAT
Parsing /home/dyerke/Documents/DSE/DSE220/HW4_Final/data/Year11/beer_PANEL_MK_1635_16

,PANID,WEEK,UNITS,OUTLET,DOLLARS,IRI_KEY,COLUPC,START_TIME_EPOCH_S,END_TIME_EPOCH_S,TRANSACTION_TIME_EPOCH_S,MONTH,DAY,YEAR,HOUR_OF_DAY,MINUTE,DAYOFWEEK
0,3369595,1479,4,GK,10,257871,0018417375121,1199059200,1199577600,1199092980,12,31,2007,9,23,0
1,3834697,1479,1,GK,8.99,257871,0013410017636,1199059200,1199577600,1199094480,12,31,2007,9,48,0
2,3308189,1479,1,GK,14.68,9999690,0011820053168,1199059200,1199577600,1199095200,12,31,2007,10,0,0
3,3812222,1479,1,GK,10.98,9999691,0013410057340,1199059200,1199577600,1199095440,12,31,2007,10,4,0
4,3356188,1479,1,GK,8.89,9999691,0017199030069,1199059200,1199577600,1199096700,12,31,2007,10,25,0
5,3315523,1479,1,GK,14.68,9999690,0013410057306,1199059200,1199577600,1199097180,12,31,2007,10,33,0
6,3341818,1479,1,GK,10.99,257871,0011820096418,1199059200,1199577600,1199098380,12,31,2007,10,53,0
7,3334490,1479,1,GK,7.49,9999691,0028066095605,1199059200,1199577600,1199099460,12,31,2007,11,11,0
8,3358846,1479,1,GK,6.19,9999690,0018417330130,1199059200,1199577600,1199099520,12,31,2007,11,12,0
9,3358846,1479,1,GK,5.99,9999690,0018417332130,1199059200,1199577600,1199099520,12,31,2007,11,12,0


In [5]:
s_outlet= master_panel_df["OUTLET"]
s_outlet_counts= s_outlet.value_counts()
s_outlet_counts

GK     67891
DK        26
MK        25
KK        23
dtype: int64

In [6]:
# decompose outlet categories to 0=Grocery, 1=Drug, 2=Mass
mapping= {
    "GK":0,
    "DK":1,
    "KK":1,
    "MK":2,
}
master_panel_df["OUTLET"]= s_outlet.apply(lambda x: mapping[x.strip()])
master_panel_df

,PANID,WEEK,UNITS,OUTLET,DOLLARS,IRI_KEY,COLUPC,START_TIME_EPOCH_S,END_TIME_EPOCH_S,TRANSACTION_TIME_EPOCH_S,MONTH,DAY,YEAR,HOUR_OF_DAY,MINUTE,DAYOFWEEK
0,3369595,1479,4,0,10,257871,0018417375121,1199059200,1199577600,1199092980,12,31,2007,9,23,0
1,3834697,1479,1,0,8.99,257871,0013410017636,1199059200,1199577600,1199094480,12,31,2007,9,48,0
2,3308189,1479,1,0,14.68,9999690,0011820053168,1199059200,1199577600,1199095200,12,31,2007,10,0,0
3,3812222,1479,1,0,10.98,9999691,0013410057340,1199059200,1199577600,1199095440,12,31,2007,10,4,0
4,3356188,1479,1,0,8.89,9999691,0017199030069,1199059200,1199577600,1199096700,12,31,2007,10,25,0
5,3315523,1479,1,0,14.68,9999690,0013410057306,1199059200,1199577600,1199097180,12,31,2007,10,33,0
6,3341818,1479,1,0,10.99,257871,0011820096418,1199059200,1199577600,1199098380,12,31,2007,10,53,0
7,3334490,1479,1,0,7.49,9999691,0028066095605,1199059200,1199577600,1199099460,12,31,2007,11,11,0
8,3358846,1479,1,0,6.19,9999690,0018417330130,1199059200,1199577600,1199099520,12,31,2007,11,12,0
9,3358846,1479,1,0,5.99,9999690,0018417332130,1199059200,1199577600,1199099520,12,31,2007,11,12,0


In [8]:
master_panel_df.to_csv("master_panel.csv")